### Imports

In [24]:
import pandas as pd
import surprise
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from surprise import accuracy
import pickle

In [2]:
links = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/links.csv')

In [3]:
movies = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/movies.csv')

In [4]:
ratings = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/ratings.csv')

In [5]:
tags = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/tags.csv')

In [6]:
ratings = ratings.drop('timestamp', axis=1)

In [7]:
# merged all data frames together
df = pd.read_csv('/Users/gabrielwarner/Data-Science/Projects/phase_4/movies_2.csv')

### Surprise Preprocessing

In [8]:
reader = surprise.Reader(rating_scale = (4., 5.))

In [9]:
data = surprise.Dataset.load_from_df(ratings, reader)

In [10]:
trainset, testset = train_test_split(data, test_size=0.2)

### Suprise with grid search

In [11]:
param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 6, 7, 8, 9, 10, 11], 'lr_all': [0.002, .003, .004, 0.005],
               'reg_all': [.02, 0.4, .5, 0.6]}
gs = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 28.9min
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed: 30.6min finished


In [14]:
svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.1547
1.1546672240307976


### Pickle

In [33]:
# Save the trained model as a pickle string.
saved_model = pickle.dumps(gs)
 
# Load the pickled model
gs_from_pickle = pickle.loads(saved_model)
 
# Use the loaded pickled model to make predictions
#gs_from_pickle.predict(trainset)

### Making Predictions

In [15]:
def get_movie_id(movie_title):
    
    # Filter Movies dataframe for movie_title
    movie_title_df = movies[['title', 'movieId']]
    # Get corresponding id from row
    movie_id = movies[movies["title"] == movie_title]['movieId'].values[0]

    # Return movie_id of movie_title
    return movie_id

In [16]:
get_movie_id("Waiting to Exhale (1995)")

4

In [17]:
# example df and inputs
t = pd.DataFrame([[1, 5, 4]], columns=['u', 'i', 'rating'])
t

,u,i,rating
0,1,5,4


In [18]:
data = surprise.Dataset.load_from_df(t, reader)

In [23]:
gs(data)

TypeError: 'GridSearchCV' object is not callable

In [20]:
def model_function(u, i, rating):
#  user ranks 5 movies
    info = {'u':[u],
       'i':[i],
       'rating':[rating]}
# convert to suprise df
    data = surprise.Dataset.load_from_df(info, reader)
    print(data)
# run through model
    gs(data)


In [21]:
model_function(1,5,4)

AttributeError: 'dict' object has no attribute 'itertuples'